<a href="https://colab.research.google.com/github/vanryuji/sqlalchemy/blob/master/tutorial/sqlalchemy_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sqlalchemy

In [0]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [0]:
### Connecting ###

engine = create_engine('sqlite:///:memory:', echo=True)

In [4]:
### Creating session ###

session = scoped_session(sessionmaker(bind=engine))
print('autocommit:', session.autocommit)

autocommit: False


In [0]:
### Declare mapping ###

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(20))
    fullname = Column(String(20))
    nickname = Column(String(20))

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)


In [6]:
### Create schema ###

print('before creating, tables:', engine.table_names())
Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(engine)
print('after creating, tables:', engine.table_names())

2019-05-28 02:57:40,669 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-28 02:57:40,671 INFO sqlalchemy.engine.base.Engine ()
2019-05-28 02:57:40,675 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-28 02:57:40,676 INFO sqlalchemy.engine.base.Engine ()
2019-05-28 02:57:40,679 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-05-28 02:57:40,681 INFO sqlalchemy.engine.base.Engine ()
before creating, tables: []
2019-05-28 02:57:40,685 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-28 02:57:40,687 INFO sqlalchemy.engine.base.Engine ()
2019-05-28 02:57:40,690 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-28 02:57:40,692 INFO sqlalchemy.engine.base.Engine ()
2019-05-28 02:57:40,694 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR(20), 
	full

In [7]:
### Insert ###

ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)
session.commit()
our_user = session.query(User).filter_by(name='ed').all()
print(our_user)

2019-05-28 02:57:40,726 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-28 02:57:40,735 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-05-28 02:57:40,740 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2019-05-28 02:57:40,744 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-28 02:57:40,748 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-28 02:57:40,751 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-28 02:57:40,753 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]


In [8]:
### Rollback ###

ed_user = User(name='ed', fullname='Ed Mickey', nickname='edsnickname')
session.add(ed_user)
session.rollback()
users = session.query(User).filter_by(name='ed').all()
print(users)

2019-05-28 02:57:40,767 INFO sqlalchemy.engine.base.Engine ROLLBACK
2019-05-28 02:57:40,770 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-28 02:57:40,772 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-28 02:57:40,774 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]


In [9]:
### Building a Relationship ###

class Address(Base):
    __tablename__ = 'addresses'

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")
    
Base.metadata.create_all(engine)
print('tables:', engine.table_names())

2019-05-28 02:57:40,805 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-28 02:57:40,808 INFO sqlalchemy.engine.base.Engine ()
2019-05-28 02:57:40,809 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-05-28 02:57:40,811 INFO sqlalchemy.engine.base.Engine ()
2019-05-28 02:57:40,813 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-05-28 02:57:40,814 INFO sqlalchemy.engine.base.Engine ()
2019-05-28 02:57:40,815 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-28 02:57:40,817 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-05-28 02:57:40,818 INFO sqlalchemy.engine.base.Engine ()
tables: ['addresses', 'users']


In [10]:
### Join ###

jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')
print('jack addresses:', jack.addresses)

jack.addresses = [Address(email_address='jack@google.com'),
                  Address(email_address='j25@yahoo.com')]
session.add(jack)
session.commit()
user = session.query(User).filter_by(name='jack').one()
print(user.addresses)

jack addresses: []
2019-05-28 02:57:40,837 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-05-28 02:57:40,839 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'gjffdd')
2019-05-28 02:57:40,842 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-05-28 02:57:40,843 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 2)
2019-05-28 02:57:40,845 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-05-28 02:57:40,847 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 2)
2019-05-28 02:57:40,848 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-28 02:57:40,850 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-28 02:57:40,852 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-28